In [4]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from IPython.display import HTML

In [5]:
lis2 = list(range(1, 20))
lis = ["page-" + str(s) for s in lis2]


In [6]:
def get_text(x):
    return x.text

In [7]:
results = []

for x in lis:
    
    find_in = 'https://internshala.com/internships/computer%20vision,data%20science,image%20processing,machine%20learning,python%2Fdjango-internship-in-bangalore,chandigarh,delhi,gurgaon,jalandhar,mumbai,navi%20mumbai,pune,thane/'
    url = find_in + x
    print(x)
    
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    internships = soup.findAll('div', {'class': 'individual_internship'})
    
    if soup.find('div', {'class': 'no_result_message'}) is None:
        results += internships
        continue
    else:
        break
    

page-1
page-2
page-3
page-4


In [8]:
df = pd.DataFrame(columns=['Location', 'Stipend_Lower', 'Stipend_Upper', 'Granularity', 'Company', 'Duration', 'Link'])

for i, item in enumerate(results):
    
    link = item.findAll('h4')[0].find('a')['href']
    website = 'https://internshala.com'
    link = '<a href=' + website + link + '> Link </a>'
    
    
    company = item.findAll('h4')[1].find('a').text
    
    details = item.find('div', {'class', 'individual_internship_details'})
    
    locations = details.find('p').findAll('span')[1].findAll('a')
    locations = list(map(get_text, locations))
    
    td = details.findAll('td')
    
    start_date = td[0].find('div').text
    duration = td[1].text
    
    #################### STIPEND ####################
    val = td[2].text.strip().split()
    if val[0] == 'Performance':
        continue
    if val[0] == 'Unpaid':
        continue
    elif '-' in val[0]:
        
        stipend = []
        stipend.append(int(val[0].split('-')[0]))
        stipend.append(int(val[0].split('-')[1]))
    else:
        stipend = int(val[0])
        
    granularity = val[-1]
    
    ################## END STIPEND ##################
    
    post_date = td[3].text
    apply_by = td[4].text
    
    # df append
    if type(stipend) == int:
        df.loc[i] = [locations, stipend, stipend, granularity, company, duration, link]
    else:
        df.loc[i] = [locations, stipend[0], stipend[1], granularity, company, duration, link]
    